In [3]:

import pandas as pd
from styleframe import StyleFrame
from openpyxl.styles import PatternFill
from openpyxl import Workbook

#open the excel workbook and save the diagnosis cat sheet as "diagnosis_cat"
file_path = "problems-diagnosis_cat-comorbidities.xlsx"
# file_path = "/Users/avikogan/Downloads/test.xlsx"


##שומרות כל גיליון תחת משתנה
diagnosis_cat = StyleFrame.read_excel(path=file_path,
                                      read_style=True,
                                      sheet_name='diagnosis_cat')

comorbidities = StyleFrame.read_excel(path=file_path,
                                      read_style=True,
                                      sheet_name='comorbidities')

patients_demo = StyleFrame.read_excel(path=file_path,
                                      read_style=True,
                                      sheet_name='patients demo')

#open 2 dictionarys that in both of them the key will be thet color and the 
#values will be in the first, the category and in the other list of all medical problems 
#for examlpe: in the colors_to_category dict we will have yellow as key and medical as value,
# and in the colors_to_set dict we will have yellow as key and a list that contain medical 
#comditions like BPH, Dyslipidemia, Hyponatremia etc...
comorbidities_to_category = dict()  # for example: s -> Sepsis
comorbidities_to_set = dict()  # for example: s -> {disease1, disease2..}

colors_to_category = dict()
colors_to_set = dict()

#diagnosis_cat["ProblemText"] = diagnosis_cat["ProblemText"].dropna()
#empty.to_excel("/Ariel/Final_Project/empty.xlsx")

#creating a set of lengeds and a set of comorbidities
for cell in diagnosis_cat["legend"]:
    colors_to_category[cell.style.bg_color] = cell.value
    colors_to_set[cell.style.bg_color] = set()

for cell in comorbidities["comorbidities"].str.lower():
    if pd.isna(cell):
        break
    else:
        comorbidities_to_category[cell[0]] = cell
        comorbidities_to_set[cell[0]] = set()
#print(colors_to_category)
#print(colors_to_set)

# print(df["ProblemText"])
for cell in diagnosis_cat["ProblemText"]:
    colors_to_set[cell.style.bg_color].add(cell.value)

# defining the sheets of the output file:
#   main_output_s - sheet with the main mission
#   problemText_comorbidity_s - sheet that separate the comorbidities letters
#       of each poblemText to separate line to allow filtering according to
#       comorbidity
#   problemText_Non_comorbidity_s - sheet that separate the comorbidities
#       letters  that not appear in the given comorbidities of each
#       poblemText to separate line to allow filtering according to comorbidity
#   ids_with_multiple_cat_g1 - patients id's that have more than one
#       category (legend) in group 1 after grouping according to the given
#       rules
#   ids_with_multiple_cat_g2 - patients id's that have more than one
#       category (legend) in group 2 after grouping according to the given
#       rules

#creat new data frame to enter the wanted flat data
output = Workbook()
main_output_s = output.active
main_output_s.title = 'main output'

first_cols = {'id': 1, 'past_legend': 2, 'current_legend': 3}

comorbidities_cols = list(comorbidities_to_category.values())
comorbidities_cols.sort()

start_g1_comorbidities_col = len(first_cols) + 1
end_g1_comorbidities_col = len(first_cols) + len(comorbidities_cols)
comorbidities_cols_g1 = dict(zip(comorbidities_cols, range(start_g1_comorbidities_col, end_g1_comorbidities_col + 1)))

start_g2_comorbidities_col = end_g1_comorbidities_col + 1
end_g2_comorbidities_col = start_g2_comorbidities_col + len(comorbidities_cols)

comorbidities_cols_g2 = dict(zip(comorbidities_cols, range(start_g2_comorbidities_col, end_g2_comorbidities_col + 1)))

#creating (and filling later on) sheets with diff types of problems
problemText_comorbidity_s = output.create_sheet('problemText comorbidity')
problemText_Non_comorbidity_s = output.create_sheet(
    'problemText Non comorbidity')
ids_with_multiple_cat_g1_s = output.create_sheet(
    'ids with multiple cat past')
ids_with_multiple_cat_g2_s = output.create_sheet(
    'ids with multiple cat present')

# first line of problemText_Non_comorbidity_s
problemText_Non_comorbidity_s.cell(row=1, column=1, value="ProblemText")
problemText_Non_comorbidity_s.cell(row=1, column=2, value="letter")
problemText_Non_comorbidity_s.cell(row=1, column=3, value="line")

# first line of problemText_comorbidity_s
problemText_comorbidity_s.cell(row=1, column=1, value="ProblemText")
problemText_comorbidity_s.cell(row=1, column=2, value="letter")

row_p_c_s = 2
row_p_N_c_s = 2

for i in range(comorbidities["ProblemText"].size):
    for letter in str(comorbidities["comorbidies"][i].value):
        if letter == "0" or letter == " ":
            continue
        if letter in comorbidities_to_set.keys():
            comorbidities_to_set[letter].add(comorbidities['ProblemText'][
                                                  i].value)
            problemText_comorbidity_s.cell(row=row_p_c_s, column=1,
                                           value=comorbidities[
                                               'ProblemText'][i].value)
            problemText_comorbidity_s.cell(row=row_p_c_s, column=2,
                                           value=letter)
            row_p_c_s += 1
        else:
            problemText_Non_comorbidity_s.cell(row=row_p_N_c_s, column=1,
                                               value=comorbidities[
                                                   'ProblemText'][i].value)
            problemText_Non_comorbidity_s.cell(row=row_p_N_c_s, column=2,
                                               value=letter)
            problemText_Non_comorbidity_s.cell(row=row_p_N_c_s, column=3,
                                               value=i)
            row_p_N_c_s += 1

#group by patients id 
id_grouby = patients_demo.groupby(['id_demo'])


def get_id_legends(colors_to_set, id_ProblemText):
    id_legends_set = set()
    for problemText in id_ProblemText:
        for color in colors_to_set.keys():
            if problemText in colors_to_set[color]:
                id_legends_set.add(colors_to_category[color])
    return id_legends_set

#כללי החלטה לפי אורית והרופאים בכדי להגיע ללג'נד אחד סופי
#לדוגמא כלל החלטה ראשון אומר שכל מה שמכיל טראומה הוא יחשב כטראומה
#כלל נוסף אומר שאם יש "סרגיקל" וגם "מדיקל" אז ההחלטה הסופית צריכה להיות "מדיקל"
def get_legends_to_write(id_legends_set):
    if "TRAUMA" in id_legends_set:
        id_legends_set = {"TRAUMA"}
        return id_legends_set
    if "SURGICAL" in id_legends_set and "MEDICAL" in id_legends_set:
        id_legends_set.remove("MEDICAL")
    if "TRANSPLANTATION" in id_legends_set and "MEDICAL" in id_legends_set:
        id_legends_set.remove("MEDICAL")
    if "OBSTETRICS" in id_legends_set and "MEDICAL" in id_legends_set:
        id_legends_set.remove("MEDICAL")
    if "TRANSPLANTATION" in id_legends_set and "SURGICAL" in id_legends_set:
        id_legends_set.remove("SURGICAL")
    if "OBSTETRICS" in id_legends_set and "SURGICAL" in id_legends_set:
        id_legends_set.remove("SURGICAL")

    return id_legends_set


def write_legends(legends_to_write, row, col, pattern, output):
    to_write = ""
    if len(legends_to_write) > 1:
        for legend in legends_to_write:
            to_write += legend + ", "
        
        to_write = to_write[:len(to_write) - 2]
    else: # 0 or 1 legend
        for legend in legends_to_write:
            to_write += legend
            
    output.cell(row=row, column=col, value=to_write)            
    output.cell(row=row, column=col).fill = pattern
    return col


def write_to_multiple_legends(id, id_legends_set, id_ProblemText,
                              output, s_row):

    output.cell(row=s_row, column=1, value=id)
    col = 3
    for legend in id_legends_set:
        output.cell(row=s_row, column=col, value=legend)
        col += 1

    col += 1  # creates gap between the legends and the problems
    for problemText in id_ProblemText:
        output.cell(row=s_row, column=col, value=problemText.value)
        col += 1




#write columns names in the first line of the main output

def write_cols_from_dict(output_s, dic):
    for col_name, col_number in dic.items():
        output_s.cell(row=1, column=col_number, value=col_name)
        
write_cols_from_dict(main_output_s, first_cols)
write_cols_from_dict(main_output_s, comorbidities_cols_g1)
write_cols_from_dict(main_output_s, comorbidities_cols_g2)


#we will start in a loop and go over each id 
row = 2

g1_s_row = 1
g2_s_row = 1

def init_comorbidities_row(output_s, row_num):
    for i in range(start_g1_comorbidities_col, end_g2_comorbidities_col + 1):
        output_s.cell(row=row_num, column=i, value=0)
    
    
for id, group in id_grouby:
    #for the first id, in the output (the data frame), put in row number by the row variable 
    #current value (starts in 0) and column 0 the patient id.
    #the first column in each row will be the relevant id.
    init_comorbidities_row(main_output_s, row)
    
    main_output_s.cell(row=row, column=first_cols['id'], value=id.value)
    
#check and separate the problem group ( 1 which is past, or 2 which is present) 
    id_ProblemText_1 = group.loc[group["ProblemGroup"] == 1]['ProblemText']
    id_ProblemText_2 = group.loc[group["ProblemGroup"] == 2]['ProblemText']

    #loop the problem text 1, loop the colors to set dict. that way we will have one problem and 
    #one color, look for the color as a key in the colors to set and from there take the list with medical problem. 
    #check if the problem is in the list, if yes put the problem in the df in the relevant place by varibles row and col,
    #and update col value +1.if no, search the problem in the next lust by the next color key...

    pattern = PatternFill(start_color='00FFFFCC', end_color='00FFFFCC',
                          fill_type='solid')

    id_legends_set = get_id_legends(colors_to_set, id_ProblemText_1)
    legends_to_write = get_legends_to_write(id_legends_set)
    if len(legends_to_write) > 1:
        write_to_multiple_legends(id.value, legends_to_write, id_ProblemText_1,
                                  ids_with_multiple_cat_g1_s, g1_s_row)
        g1_s_row += 1
        
    write_legends(legends_to_write, row, first_cols['past_legend'], pattern, main_output_s)

#same as before but only on problem text 2
    pattern = PatternFill(start_color='00CC99FF', end_color='00CC99FF',
                          fill_type='solid')

    id_legends_set = get_id_legends(colors_to_set, id_ProblemText_2)
    legends_to_write = get_legends_to_write(id_legends_set)
    if len(legends_to_write) > 1:
        write_to_multiple_legends(id.value, legends_to_write, id_ProblemText_2,
                                  ids_with_multiple_cat_g2_s, g2_s_row)
        g2_s_row += 1
        
    write_legends(legends_to_write, row, first_cols['current_legend'], pattern, main_output_s)

    pattern = PatternFill(start_color='00FF0000', end_color='00FF0000',
                          fill_type='solid')
    
    for problemText in id_ProblemText_1:
        for comorbiditie in comorbidities_to_set.keys():
            if problemText in comorbidities_to_set[comorbiditie]:
                
                comorbidities_g1_col_num = comorbidities_cols_g1[comorbidities_to_category[comorbiditie]]
                new_val = main_output_s.cell(row=row, column=comorbidities_g1_col_num).value + 1
                
                main_output_s.cell(row=row, column=comorbidities_g1_col_num, value=new_val)
                main_output_s.cell(row=row, column=comorbidities_g1_col_num).fill = pattern


    pattern = PatternFill(start_color='0000FF00', end_color='0000FF00',
                          fill_type='solid')

    for problemText in id_ProblemText_2:
        for comorbiditie in comorbidities_to_set.keys():
            if problemText in comorbidities_to_set[comorbiditie]:
                
                comorbidities_g2_col_num = comorbidities_cols_g2[comorbidities_to_category[comorbiditie]]
                new_val = main_output_s.cell(row=row, column=comorbidities_g2_col_num).value + 1
                
                main_output_s.cell(row=row, column=comorbidities_g2_col_num, value=new_val)
                main_output_s.cell(row=row, column=comorbidities_g2_col_num).fill = pattern

    row += 1



output.save("output_como.xlsx")

ModuleNotFoundError: No module named 'styleframe'

In [ ]:
## output_como_changed_id.xlsx